In [8]:
#@title 🎯 Fukuoka MQ Challenge Solver - COMPLETE with .bz2 Support
#@markdown ## Full solver for MQ Challenge - Supports .bz2, .gz, .xz files
#@markdown ## Quaternion Dynamics + PySAT Hybrid Algorithm

!pip install python-sat requests -q

import numpy as np
import lzma
import gzip
import bz2  # <-- ADDED BZ2 SUPPORT!
import math
import time
import requests
import os
import re
from io import BytesIO
from google.colab import files
from pysat.solvers import Solver as PySolver
from collections import defaultdict

print("✓ All libraries loaded successfully!")

# ============================================================
# STABLE QUATERNION CORE
# ============================================================
class Quaternion:
    """Stable quaternion implementation for SAT heuristics"""
    __slots__ = ['q']

    def __init__(self, w=0.0, x=0.0, y=0.0, z=0.0):
        self.q = np.array([w, x, y, z], dtype=np.float64)
        self._stabilize()

    def _stabilize(self):
        self.q = np.clip(self.q, -1e15, 1e15)
        self.q = np.nan_to_num(self.q, nan=0.0, posinf=1e10, neginf=-1e10)
        max_val = np.max(np.abs(self.q))
        if max_val > 1e8:
            self.q /= max_val

    @property
    def w(self): return self.q[0]
    @property
    def x(self): return self.q[1]
    @property
    def y(self): return self.q[2]
    @property
    def z(self): return self.q[3]

    def __add__(self, other):
        r = Quaternion()
        r.q = self.q + other.q
        r._stabilize()
        return r

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            r = Quaternion()
            r.q = self.q * other
            r._stabilize()
            return r

        a, b, c, d = self.q
        e, f, g, h = other.q

        r = Quaternion()
        r.q = np.array([
            a*e - b*f - c*g - d*h,
            a*f + b*e + c*h - d*g,
            a*g - b*h + c*e + d*f,
            a*h + b*g - c*f + d*e
        ], dtype=np.float64)
        r._stabilize()
        return r

    def norm(self):
        return np.sqrt(np.sum(self.q ** 2))

    def to_matrix(self):
        w, x, y, z = self.q
        return np.array([
            [w, -x, -y, -z],
            [x,  w, -z,  y],
            [y,  z,  w, -x],
            [z, -y,  x,  w]
        ], dtype=np.float64)


# ============================================================
# FILE DECOMPRESSION HANDLER - SUPPORTS ALL FORMATS
# ============================================================
def decompress_file(content, filename):
    """
    Decompress file based on extension
    Supports: .bz2, .gz, .xz, and plain text
    """
    print(f"  Processing: {filename}")
    print(f"  Raw size: {len(content)} bytes")

    try:
        # BZ2 compression (Fukuoka MQ Challenge format)
        if filename.endswith('.bz2'):
            print("  Decompressing .bz2...")
            decompressed = bz2.decompress(content)
            text = decompressed.decode('utf-8', errors='ignore')
            print(f"  Decompressed: {len(text)} characters")
            return text

        # GZIP compression
        elif filename.endswith('.gz'):
            print("  Decompressing .gz...")
            decompressed = gzip.decompress(content)
            text = decompressed.decode('utf-8', errors='ignore')
            print(f"  Decompressed: {len(text)} characters")
            return text

        # XZ/LZMA compression
        elif filename.endswith('.xz') or filename.endswith('.lzma'):
            print("  Decompressing .xz/.lzma...")
            decompressed = lzma.decompress(content)
            text = decompressed.decode('utf-8', errors='ignore')
            print(f"  Decompressed: {len(text)} characters")
            return text

        # Plain text
        else:
            print("  Reading as plain text...")
            if isinstance(content, bytes):
                text = content.decode('utf-8', errors='ignore')
            else:
                text = content
            return text

    except Exception as e:
        print(f"  ERROR decompressing: {str(e)}")
        # Try to read as plain text anyway
        if isinstance(content, bytes):
            return content.decode('utf-8', errors='ignore')
        return content


# ============================================================
# MQ PROBLEM PARSER (Fukuoka Format)
# ============================================================
class MQProblem:
    """
    Parser for Fukuoka MQ Challenge format
    Supports all challenge types: I, II, III, IV, V, VI

    Format description from mqchallenge.org:
    - Galois Field: GF(2) or GF(2^k)
    - Variables: n
    - Equations: m
    - Coefficients in specific order
    """

    def __init__(self):
        self.n = 0  # Number of variables
        self.m = 0  # Number of equations
        self.field = 2  # GF(2) by default
        self.equations = []  # List of equations
        self.seed = None
        self.challenge_type = None
        self.raw_coefficients = []  # Raw coefficient data

    def parse_challenge_file(self, content, filename=""):
        """Parse MQ Challenge format file"""
        print(f"\n  Parsing MQ Challenge file: {filename}")

        lines = content.strip().split('\n')

        self.equations = []
        self.raw_coefficients = []

        # Detect challenge type from filename
        filename_lower = filename.lower()
        if 'typei' in filename_lower or 'type1' in filename_lower or 'type_i' in filename_lower:
            if 'typeii' in filename_lower or 'type_ii' in filename_lower:
                self.challenge_type = 'II'
            elif 'typeiii' in filename_lower or 'type_iii' in filename_lower:
                self.challenge_type = 'III'
            elif 'typeiv' in filename_lower or 'type_iv' in filename_lower:
                self.challenge_type = 'IV'
            else:
                self.challenge_type = 'I'
        elif 'typev' in filename_lower or 'type5' in filename_lower or 'type_v' in filename_lower:
            if 'typevi' in filename_lower or 'type_vi' in filename_lower:
                self.challenge_type = 'VI'
            else:
                self.challenge_type = 'V'

        # Parse header and content
        in_header = True
        coefficient_lines = []

        for line in lines:
            line = line.strip()

            # Skip empty lines
            if not line:
                continue

            # Parse comments/header
            if line.startswith('#') or line.startswith('c') or line.startswith('*'):
                # Extract info from comments
                line_lower = line.lower()

                if 'galois' in line_lower or 'gf(' in line_lower:
                    match = re.search(r'gf\s*\(\s*(\d+)\s*\)', line_lower)
                    if match:
                        self.field = int(match.group(1))

                elif 'variable' in line_lower:
                    match = re.search(r'(\d+)', line)
                    if match:
                        self.n = int(match.group(1))

                elif 'polynomial' in line_lower or 'equation' in line_lower:
                    match = re.search(r'(\d+)', line)
                    if match:
                        self.m = int(match.group(1))

                elif 'seed' in line_lower:
                    match = re.search(r'(\d+)', line)
                    if match:
                        self.seed = int(match.group(1))
                continue

            # Try to parse as header line (key : value format)
            if ':' in line and in_header:
                key, _, value = line.partition(':')
                key = key.strip().lower()
                value = value.strip()

                if 'galois' in key or 'field' in key:
                    match = re.search(r'\d+', value)
                    if match:
                        self.field = int(match.group())
                elif 'variable' in key:
                    match = re.search(r'\d+', value)
                    if match:
                        self.n = int(match.group())
                elif 'polynomial' in key or 'equation' in key:
                    match = re.search(r'\d+', value)
                    if match:
                        self.m = int(match.group())
                elif 'seed' in key:
                    match = re.search(r'\d+', value)
                    if match:
                        self.seed = int(match.group())
                continue

            # Not a header line - start parsing coefficients
            in_header = False

            # Check if line contains polynomial notation
            if '+' in line or '*' in line or 'x' in line.lower():
                # Polynomial format
                parsed_eq = self._parse_polynomial_line(line)
                if parsed_eq:
                    self.equations.append(parsed_eq)
            else:
                # Coefficient format (space or comma separated numbers)
                coeffs = self._extract_coefficients(line)
                if coeffs:
                    coefficient_lines.append(coeffs)

        # If we got coefficient lines, convert to equations
        if coefficient_lines and not self.equations:
            self._convert_coefficients_to_equations(coefficient_lines)

        # Validate
        if self.n == 0 and self.equations:
            # Try to infer n from equations
            max_var = 0
            for eq in self.equations:
                for (i, j) in eq.get('quadratic', []):
                    max_var = max(max_var, i, j)
                for i in eq.get('linear', []):
                    max_var = max(max_var, i)
            self.n = max_var

        if self.m == 0:
            self.m = len(self.equations)

        print(f"  ✓ Parsed MQ Problem:")
        print(f"    Field: GF({self.field})")
        print(f"    Variables (n): {self.n}")
        print(f"    Equations (m): {self.m}")
        print(f"    Parsed equations: {len(self.equations)}")
        if self.challenge_type:
            print(f"    Challenge Type: {self.challenge_type}")
        if self.seed:
            print(f"    Seed: {self.seed}")

        return len(self.equations) > 0

    def _extract_coefficients(self, line):
        """Extract coefficients from a line"""
        # Remove common separators and extract numbers
        line = line.replace(',', ' ').replace(';', ' ').replace('\t', ' ')

        coeffs = []
        for token in line.split():
            try:
                # Handle hex format
                if token.startswith('0x'):
                    coeffs.append(int(token, 16))
                else:
                    coeffs.append(int(token))
            except ValueError:
                continue

        return coeffs if coeffs else None

    def _parse_polynomial_line(self, line):
        """Parse a polynomial equation line"""
        equation = {
            'quadratic': [],
            'linear': [],
            'constant': 0
        }

        # Clean the line
        line = line.replace(' ', '').replace('\t', '').replace(';', '')

        # Handle equation format: LHS = RHS
        if '=' in line:
            lhs, rhs = line.split('=', 1)
            # Move RHS constant to equation
            try:
                rhs_val = int(rhs.strip())
                if rhs_val == 1:
                    equation['constant'] = 1
            except:
                pass
            line = lhs

        # Split by +
        terms = re.split(r'\+', line)

        for term in terms:
            term = term.strip()
            if not term:
                continue

            # Constant term
            if term == '1':
                equation['constant'] ^= 1
                continue
            if term == '0':
                continue

            # Find variables in term
            var_matches = re.findall(r'x_?(\d+)', term, re.IGNORECASE)

            if len(var_matches) >= 2:
                # Quadratic term
                i, j = int(var_matches[0]), int(var_matches[1])
                equation['quadratic'].append((min(i,j), max(i,j)))
            elif len(var_matches) == 1:
                # Check if it's x*x (same variable squared)
                if term.count('x') >= 2 or term.count('X') >= 2 or '*' in term:
                    # x_i * x_i = x_i in GF(2)
                    i = int(var_matches[0])
                    equation['linear'].append(i)
                else:
                    # Linear term
                    i = int(var_matches[0])
                    equation['linear'].append(i)

        return equation if (equation['quadratic'] or equation['linear'] or equation['constant']) else None

    def _convert_coefficients_to_equations(self, coef_lines):
        """
        Convert coefficient matrix to equations

        MQ Challenge coefficient order for n variables:
        - Quadratic: x1*x1, x1*x2, ..., x1*xn, x2*x2, x2*x3, ..., xn*xn
        - Linear: x1, x2, ..., xn
        - Constant: 1

        Total coefficients per equation: n*(n+1)/2 + n + 1
        """
        if not self.n:
            # Try to infer n from coefficient count
            if coef_lines:
                num_coeffs = len(coef_lines[0])
                # Solve: n*(n+1)/2 + n + 1 = num_coeffs
                # n^2/2 + 3n/2 + 1 = num_coeffs
                # n^2 + 3n + 2 - 2*num_coeffs = 0
                # Using quadratic formula
                discriminant = 9 - 4*(2 - 2*num_coeffs)
                if discriminant >= 0:
                    n = int((-3 + np.sqrt(discriminant)) / 2)
                    if n > 0:
                        self.n = n
                        print(f"    Inferred n={n} from {num_coeffs} coefficients")

        if not self.n:
            print("    WARNING: Could not determine number of variables")
            return

        n = self.n
        expected_coeffs = n * (n + 1) // 2 + n + 1

        for coeffs in coef_lines:
            equation = {
                'quadratic': [],
                'linear': [],
                'constant': 0
            }

            idx = 0

            # Quadratic terms: x_i * x_j for i <= j
            for i in range(1, n + 1):
                for j in range(i, n + 1):
                    if idx < len(coeffs) and coeffs[idx] == 1:
                        if i == j:
                            # x_i^2 = x_i in GF(2)
                            if i not in equation['linear']:
                                equation['linear'].append(i)
                        else:
                            equation['quadratic'].append((i, j))
                    idx += 1

            # Linear terms
            for i in range(1, n + 1):
                if idx < len(coeffs) and coeffs[idx] == 1:
                    # Toggle (XOR) since x_i^2 might have added it
                    if i in equation['linear']:
                        equation['linear'].remove(i)
                    else:
                        equation['linear'].append(i)
                idx += 1

            # Constant term
            if idx < len(coeffs):
                equation['constant'] = coeffs[idx] % 2

            self.equations.append(equation)

        self.m = len(self.equations)


# ============================================================
# MQ TO SAT CONVERTER
# ============================================================
class MQtoSATConverter:
    """Convert MQ problem over GF(2) to SAT (CNF)"""

    def __init__(self, mq_problem):
        self.mq = mq_problem
        self.num_vars = 0
        self.clauses = []
        self.var_map = {}
        self.reverse_map = {}

    def convert(self):
        """Convert MQ problem to CNF clauses"""
        print("\n  Converting MQ to SAT...")

        # Original variables: x_1, ..., x_n
        for i in range(1, self.mq.n + 1):
            self._get_var('x', i)

        # Convert each equation
        for eq_idx, eq in enumerate(self.mq.equations):
            self._convert_equation(eq, eq_idx)

        print(f"    SAT Variables: {self.num_vars}")
        print(f"    SAT Clauses: {len(self.clauses)}")

        return self.clauses, self.num_vars, self.var_map, self.reverse_map

    def _get_var(self, var_type, *indices):
        """Get or create a SAT variable"""
        key = (var_type,) + indices
        if key not in self.var_map:
            self.num_vars += 1
            self.var_map[key] = self.num_vars
            self.reverse_map[self.num_vars] = key
        return self.var_map[key]

    def _convert_equation(self, eq, eq_idx):
        """Convert one equation to CNF clauses using XOR encoding"""
        terms = []

        # Quadratic terms: create AND auxiliary variables
        for (i, j) in eq['quadratic']:
            and_var = self._get_var('and', i, j)
            xi = self._get_var('x', i)
            xj = self._get_var('x', j)

            # Encode: and_var ↔ (xi ∧ xj)
            self.clauses.append((-and_var, xi))
            self.clauses.append((-and_var, xj))
            self.clauses.append((-xi, -xj, and_var))

            terms.append(and_var)

        # Linear terms
        for i in eq['linear']:
            xi = self._get_var('x', i)
            terms.append(xi)

        # Constant term
        if eq['constant'] == 1:
            const_var = self._get_var('const', eq_idx)
            self.clauses.append((const_var,))
            terms.append(const_var)

        # Encode XOR of all terms = 0
        if len(terms) == 0:
            return
        elif len(terms) == 1:
            self.clauses.append((-terms[0],))
        elif len(terms) == 2:
            a, b = terms
            self.clauses.append((a, -b))
            self.clauses.append((-a, b))
        else:
            self._encode_xor_chain(terms, eq_idx)

    def _encode_xor_chain(self, terms, eq_idx):
        """Encode XOR of multiple terms"""
        current = terms[0]

        for i in range(1, len(terms)):
            next_term = terms[i]

            if i == len(terms) - 1:
                # Last: XOR must be 0
                self.clauses.append((current, -next_term))
                self.clauses.append((-current, next_term))
            else:
                aux = self._get_var('xor', eq_idx, i)
                self.clauses.append((-aux, -current, -next_term))
                self.clauses.append((-aux, current, next_term))
                self.clauses.append((aux, -current, next_term))
                self.clauses.append((aux, current, -next_term))
                current = aux


# ============================================================
# HYBRID SAT SOLVER WITH QUATERNION HEURISTIC
# ============================================================
class SATSolverQuaternion:
    """Hybrid SAT Solver: Quaternion Heuristic + PySAT CDCL"""

    def __init__(self):
        self.num_vars = 0
        self.clauses = []
        self.var_pos = None
        self.var_neg = None
        self.var_clauses = None
        self.implications = None

    def load_clauses(self, clauses, num_vars):
        """Load clauses directly"""
        self.clauses = [tuple(c) for c in clauses]
        self.num_vars = num_vars
        self._build_structures()
        return True

    def _build_structures(self):
        """Build variable occurrence structures"""
        n = self.num_vars
        self.var_pos = np.zeros(n + 1, dtype=np.float64)
        self.var_neg = np.zeros(n + 1, dtype=np.float64)
        self.var_clauses = [[] for _ in range(n + 1)]
        self.implications = defaultdict(set)

        for idx, clause in enumerate(self.clauses):
            clause_len = len(clause)
            if clause_len == 0:
                continue
            weight = 1.0 / clause_len

            for lit in clause:
                var = abs(lit)
                if var <= n:
                    if lit > 0:
                        self.var_pos[var] += weight
                    else:
                        self.var_neg[var] += weight
                    self.var_clauses[var].append(idx)

            if clause_len == 2:
                a, b = clause
                self.implications[-a].add(b)
                self.implications[-b].add(a)

    def count_satisfied(self, assignments):
        """Count satisfied clauses"""
        count = 0
        for clause in self.clauses:
            for lit in clause:
                var = abs(lit)
                val = assignments.get(var, True)
                if (lit > 0 and val) or (lit < 0 and not val):
                    count += 1
                    break
        return count

    def _heuristic_quaternion(self):
        """Quaternion-based heuristic"""
        n_vars = self.num_vars
        n_clauses = len(self.clauses)

        assignments = {}
        forced = set()

        # Unit propagation
        for clause in self.clauses:
            if len(clause) == 1:
                lit = clause[0]
                var = abs(lit)
                assignments[var] = (lit > 0)
                forced.add(var)

        # Implication propagation
        changed = True
        passes = 0
        max_passes = min(int(np.log2(n_vars + 1)) + 5, 20)

        while changed and passes < max_passes:
            changed = False
            passes += 1

            for var in list(assignments.keys()):
                val = assignments[var]
                lit = var if val else -var

                for impl in self.implications.get(lit, []):
                    impl_var = abs(impl)
                    impl_val = (impl > 0)

                    if impl_var not in assignments:
                        assignments[impl_var] = impl_val
                        forced.add(impl_var)
                        changed = True

        # Quaternion dynamics for remaining
        remaining_vars = [v for v in range(1, n_vars + 1) if v not in assignments]

        if not remaining_vars:
            return assignments

        q_global = Quaternion(0, 0, 0, 0)

        for var in remaining_vars:
            if var >= len(self.var_pos):
                assignments[var] = True
                continue

            pos = self.var_pos[var]
            neg = self.var_neg[var]
            total = pos + neg

            if total < 1e-10:
                assignments[var] = True
                continue

            A = var % 1000  # Prevent overflow
            B = 1 if pos >= neg else -1
            C = min(len(self.var_clauses[var]) if var < len(self.var_clauses) else 0, 1000)

            n_val = A - B + C
            if n_val == 0:
                n_val = 1

            a = (pos - neg) / (total + 1)
            b = total / (n_vars + 1)

            b_power = np.exp(np.log(b + 1) / max(abs(n_val), 1)) if b > 0 else 1.0

            q = Quaternion(
                w = a + b_power,
                x = B * pos / (total + 1),
                y = neg / (total + 1),
                z = np.log(abs(n_val) + 1) / np.log(n_vars + 2)
            )

            q_global = q_global + q

        # Derive assignment scores
        n_coef = n_clauses / (n_vars + 1) + 1
        norm_g = q_global.norm()
        factor = np.log(norm_g + 1) / (n_coef + 1) if norm_g > 1e-10 else 1.0

        derivative = Quaternion(
            w = q_global.w * factor + q_global.x * 0.3,
            x = q_global.x * factor + q_global.y * 0.3,
            y = q_global.y * factor + q_global.z * 0.3,
            z = q_global.z * factor + q_global.w * 0.3
        )

        M = derivative.to_matrix()
        eigenvalues = np.linalg.eigvals(M)
        eigen_signs = np.sign(np.real(eigenvalues))
        matrix_values = M.flatten()

        for var in remaining_vars:
            if var in assignments or var >= len(self.var_pos):
                if var not in assignments:
                    assignments[var] = True
                continue

            pos = self.var_pos[var]
            neg = self.var_neg[var]
            total = pos + neg

            idx = (var - 1) % 16
            val_matrix = matrix_values[idx]
            eigen_idx = (var - 1) % 4
            val_eigen = eigen_signs[eigen_idx]

            polarity = (pos - neg) / total if total > 0 else 0

            clause_score = 0
            if var < len(self.var_clauses):
                for cl_idx in self.var_clauses[var][:100]:  # Limit for speed
                    clause = self.clauses[cl_idx]
                    for lit in clause:
                        if abs(lit) == var:
                            weight = 1.0 / len(clause)
                            clause_score += weight if lit > 0 else -weight
                            break

            score = (
                0.20 * np.sign(val_matrix) +
                0.15 * val_eigen +
                0.35 * np.sign(polarity) +
                0.30 * np.sign(clause_score)
            )

            assignments[var] = (score >= 0)

        # Fill remaining
        for v in range(1, n_vars + 1):
            if v not in assignments:
                assignments[v] = True

        return assignments

    def _pysat_solve(self, timeout=None):
        """Use PySAT for exact solving"""
        solver = PySolver(name='g3')

        for clause in self.clauses:
            solver.add_clause(list(clause))

        sat = solver.solve()

        if not sat:
            solver.delete()
            return "UNSAT", None

        model = solver.get_model()
        solver.delete()

        assignments = {}
        for lit in model:
            var = abs(lit)
            assignments[var] = (lit > 0)

        return "SAT", assignments

    def solve(self):
        """Main solving method"""
        n_vars = self.num_vars
        n_clauses = len(self.clauses)

        if n_clauses == 0:
            return "SAT", {i: True for i in range(1, n_vars + 1)}, 0, None

        print("  [1] Quaternionic approximation O(log n)...")
        start_heur = time.time()
        heur_assign = self._heuristic_quaternion()
        heur_time = time.time() - start_heur

        sat_heur = self.count_satisfied(heur_assign)
        pct_heur = sat_heur * 100.0 / n_clauses
        print(f"      Heuristic: {sat_heur}/{n_clauses} ({pct_heur:.2f}%) in {heur_time:.4f}s")

        if sat_heur == n_clauses:
            return "SAT", heur_assign, np.log(n_vars + n_clauses + 1), heur_assign

        print("  [2] Exact decision with PySAT (CDCL)...")
        result, exact_assign = self._pysat_solve()

        return result, heur_assign, np.log(n_vars + n_clauses + 1), exact_assign


# ============================================================
# FUKUOKA MQ CHALLENGE MANAGER
# ============================================================
class FukuokaMQChallenge:
    """Complete Fukuoka MQ Challenge solver"""

    def __init__(self):
        self.results = []
        self.solver = SATSolverQuaternion()

    def generate_test_problem(self, n=10, m=15, seed=42):
        """Generate a random test MQ problem"""
        np.random.seed(seed)

        mq = MQProblem()
        mq.n = n
        mq.m = m
        mq.field = 2
        mq.seed = seed

        # Random solution
        solution = np.random.randint(0, 2, n)

        for _ in range(m):
            eq = {'quadratic': [], 'linear': [], 'constant': 0}

            # Random quadratic terms
            num_quad = np.random.randint(1, min(6, n*(n-1)//2 + 1))
            used_pairs = set()
            for _ in range(num_quad):
                i = np.random.randint(1, n+1)
                j = np.random.randint(i, n+1)
                if (i, j) not in used_pairs:
                    eq['quadratic'].append((i, j))
                    used_pairs.add((i, j))

            # Random linear terms
            num_lin = np.random.randint(0, min(4, n))
            used_vars = set()
            for _ in range(num_lin):
                i = np.random.randint(1, n+1)
                if i not in used_vars:
                    eq['linear'].append(i)
                    used_vars.add(i)

            # Compute constant to satisfy solution
            value = 0
            for (i, j) in eq['quadratic']:
                value ^= (solution[i-1] & solution[j-1])
            for i in eq['linear']:
                value ^= solution[i-1]

            eq['constant'] = value
            mq.equations.append(eq)

        return mq, solution

    def solve_mq_problem(self, mq_problem, problem_name="Unknown"):
        """Solve a single MQ problem"""
        print(f"\n{'='*60}")
        print(f"  SOLVING: {problem_name}")
        print(f"{'='*60}")

        # Convert MQ to SAT
        converter = MQtoSATConverter(mq_problem)
        clauses, num_vars, var_map, reverse_map = converter.convert()

        # Load into solver
        self.solver = SATSolverQuaternion()
        self.solver.load_clauses(clauses, num_vars)

        # Solve
        print("\n  Solving SAT instance...")
        start_time = time.time()
        result, heur_assign, complexity, exact_assign = self.solver.solve()
        solve_time = time.time() - start_time

        # Extract MQ solution
        mq_solution = None
        if result == "SAT" and exact_assign:
            mq_solution = {}
            for i in range(1, mq_problem.n + 1):
                key = ('x', i)
                if key in var_map:
                    sat_var = var_map[key]
                    mq_solution[i] = exact_assign.get(sat_var, False)
                else:
                    mq_solution[i] = False

        # Verify
        verified = self._verify_mq_solution(mq_problem, mq_solution) if mq_solution else False

        sat_count = self.solver.count_satisfied(heur_assign) if heur_assign else 0

        result_data = {
            'name': problem_name,
            'n': mq_problem.n,
            'm': mq_problem.m,
            'result': result,
            'time': solve_time,
            'sat_vars': num_vars,
            'sat_clauses': len(clauses),
            'heuristic_satisfied': sat_count,
            'heuristic_percent': sat_count * 100.0 / len(clauses) if clauses else 100,
            'solution': mq_solution,
            'verified': verified
        }

        self.results.append(result_data)

        print(f"\n  {'='*50}")
        print(f"  RESULT: {result}")
        print(f"  Time: {solve_time:.4f} seconds")
        print(f"  Heuristic accuracy: {result_data['heuristic_percent']:.2f}%")
        if result == "SAT":
            print(f"  Solution verified: {'✓ YES' if verified else '✗ NO'}")
            if mq_solution and mq_problem.n <= 50:
                sol_str = ''.join([str(int(mq_solution.get(i, 0))) for i in range(1, mq_problem.n + 1)])
                print(f"  Solution: {sol_str}")
        print(f"  {'='*50}")

        return result_data

    def _verify_mq_solution(self, mq_problem, solution):
        """Verify solution satisfies all equations"""
        if not solution:
            return False

        for eq_idx, eq in enumerate(mq_problem.equations):
            value = eq['constant']

            for (i, j) in eq['quadratic']:
                vi = 1 if solution.get(i, False) else 0
                vj = 1 if solution.get(j, False) else 0
                value ^= (vi & vj)

            for i in eq['linear']:
                if solution.get(i, False):
                    value ^= 1

            if value != 0:
                return False

        return True

    def run_full_challenge(self):
        """Run complete challenge simulation"""
        print("\n" + "="*70)
        print("  🎯 FUKUOKA MQ CHALLENGE SOLVER - QUATERNION DYNAMICS")
        print("  Testing solver limits with progressive difficulty")
        print("="*70)

        challenges = [
            # (n, m, name)
            (8, 12, "Warmup-1"),
            (10, 15, "Warmup-2"),
            (12, 18, "Easy-1"),
            (14, 21, "Easy-2"),
            (16, 24, "Medium-1"),
            (18, 27, "Medium-2"),
            (20, 30, "Hard-1"),
            (22, 33, "Hard-2"),
            (24, 36, "VeryHard-1"),
            (26, 39, "VeryHard-2"),
            (28, 42, "Expert-1"),
            (30, 45, "Expert-2"),
            (32, 48, "Master-1"),
            (34, 51, "Master-2"),
            (36, 54, "Grandmaster-1"),
            (38, 57, "Grandmaster-2"),
            (40, 60, "Legendary"),
            (42, 63, "Mythic"),
            (45, 68, "Challenge"),
            (48, 72, "Extreme"),
            (50, 75, "Ultimate"),
        ]

        for n, m, name in challenges:
            print(f"\n{'▶'*60}")
            print(f"▶ CHALLENGE: {name} (n={n}, m={m})")
            print(f"{'▶'*60}")

            try:
                mq, _ = self.generate_test_problem(n, m, seed=n*100+m)
                result = self.solve_mq_problem(mq, name)

                status = "✓ PASSED" if result['result'] == "SAT" and result['verified'] else "✗ FAILED"
                print(f"\n  {status}")

                # Stop if too slow
                if result['time'] > 120:
                    print(f"\n  ⚠ Time limit reached! Max solvable: n={n}")
                    break

            except Exception as e:
                print(f"  ERROR: {str(e)}")
                import traceback
                traceback.print_exc()
                break

        self._print_summary()

    def _print_summary(self):
        """Print results summary"""
        print("\n" + "="*70)
        print("  📊 RESULTS SUMMARY")
        print("="*70)

        print(f"\n  {'Problem':<20} {'n':>4} {'m':>4} {'Result':>8} {'Time':>10} {'Heur%':>8} {'OK':>4}")
        print("  " + "-"*65)

        for r in self.results:
            verified = "✓" if r.get('verified', False) else "✗"
            print(f"  {r['name']:<20} {r['n']:>4} {r['m']:>4} {r['result']:>8} {r['time']:>10.4f} {r['heuristic_percent']:>7.1f}% {verified:>4}")

        total = len(self.results)
        solved = sum(1 for r in self.results if r['result'] == "SAT" and r.get('verified', False))
        total_time = sum(r['time'] for r in self.results)
        avg_heur = np.mean([r['heuristic_percent'] for r in self.results]) if self.results else 0

        print("\n  " + "-"*65)
        print(f"  Total: {total} | Solved: {solved} ({100*solved/total if total else 0:.1f}%)")
        print(f"  Total time: {total_time:.2f}s | Avg heuristic: {avg_heur:.1f}%")
        print("="*70)


# ============================================================
# FILE UPLOAD HANDLER
# ============================================================
def handle_upload():
    """Handle uploaded MQ Challenge files (.bz2, .gz, .xz, .txt)"""
    print("\n" + "="*60)
    print("  📁 Upload your MQ Challenge file")
    print("  Supported: .bz2, .gz, .xz, .txt, .mq")
    print("="*60 + "\n")

    uploaded = files.upload()

    if not uploaded:
        print("[ERROR] No file uploaded")
        return None

    challenge = FukuokaMQChallenge()

    for filename, content in uploaded.items():
        print(f"\n{'='*50}")
        print(f"  File: {filename}")
        print(f"{'='*50}")

        # Decompress
        text_content = decompress_file(content, filename)

        if not text_content:
            print("[ERROR] Could not read file content")
            continue

        # Show preview
        print("\n  File preview (first 500 chars):")
        print("  " + "-"*40)
        preview = text_content[:500].replace('\n', '\n  ')
        print(f"  {preview}")
        print("  " + "-"*40)

        # Parse MQ problem
        mq = MQProblem()
        if mq.parse_challenge_file(text_content, filename):
            result = challenge.solve_mq_problem(mq, filename)

            # Save result
            save_result(filename, result, mq)

            challenge._print_summary()
            return result
        else:
            print("[ERROR] Could not parse MQ problem")
            print("  Trying to extract more info from file...")

            # Show more of the file for debugging
            print("\n  Full content preview:")
            for i, line in enumerate(text_content.split('\n')[:30]):
                print(f"  {i+1:3}: {line[:80]}")

    return None


def save_result(filename, result, mq_problem):
    """Save solution file"""
    # Clean filename
    base = filename
    for ext in ['.bz2', '.gz', '.xz', '.txt', '.mq']:
        if base.endswith(ext):
            base = base[:-len(ext)]

    output_file = f"{base}_SOLUTION.txt"

    lines = [
        "# " + "="*50,
        "# MQ Challenge Solution",
        "# Quaternion Dynamics + PySAT Hybrid Solver",
        "# " + "="*50,
        f"# Problem: {filename}",
        f"# Variables (n): {mq_problem.n}",
        f"# Equations (m): {mq_problem.m}",
        f"# Field: GF({mq_problem.field})",
        f"# Result: {result['result']}",
        f"# Time: {result['time']:.6f} seconds",
        f"# Heuristic accuracy: {result['heuristic_percent']:.2f}%",
        f"# Verified: {result.get('verified', False)}",
        "#"
    ]

    if result['result'] == "SAT" and result.get('solution'):
        sol = result['solution']
        sol_bits = ''.join([str(int(sol.get(i, 0))) for i in range(1, mq_problem.n + 1)])
        lines.append(f"# Solution (binary): {sol_bits}")
        lines.append("#")
        lines.append("# Variable assignments:")

        for i in range(1, mq_problem.n + 1):
            lines.append(f"x{i} = {int(sol.get(i, 0))}")
    else:
        lines.append("# No solution found (UNSAT or timeout)")

    content = '\n'.join(lines)

    with open(output_file, 'w') as f:
        f.write(content)

    print(f"\n  💾 Solution saved: {output_file}")

    try:
        files.download(output_file)
    except:
        print(f"  (Download manually from: {output_file})")

    return output_file


# ============================================================
# MAIN MENU
# ============================================================
def main():
    print("="*70)
    print("   🎯 FUKUOKA MQ CHALLENGE SOLVER")
    print("   Quaternion Dynamics + PySAT Hybrid Algorithm")
    print("   https://www.mqchallenge.org/")
    print("="*70)

    print("""
    Select mode:

      1. 📁 Upload MQ Challenge file (.bz2, .gz, .xz, .txt)
      2. 🏃 Run full challenge simulation (progressive difficulty)
      3. ⚡ Quick test (small problem)
      4. 🎯 Custom test (specify n, m)
    """)

    mode = input("Enter mode (1/2/3/4) [default=1]: ").strip() or "1"

    if mode == "1":
        handle_upload()

    elif mode == "2":
        challenge = FukuokaMQChallenge()
        challenge.run_full_challenge()

    elif mode == "3":
        print("\n  Running quick test (n=12, m=18)...")
        challenge = FukuokaMQChallenge()
        mq, sol = challenge.generate_test_problem(12, 18, seed=42)
        challenge.solve_mq_problem(mq, "QuickTest")
        challenge._print_summary()

    elif mode == "4":
        try:
            n = int(input("Enter n (variables): "))
            m = int(input("Enter m (equations): "))
            seed = int(input("Enter seed [default=42]: ") or "42")

            challenge = FukuokaMQChallenge()
            mq, sol = challenge.generate_test_problem(n, m, seed)
            challenge.solve_mq_problem(mq, f"Custom_n{n}_m{m}")
            challenge._print_summary()
        except ValueError:
            print("[ERROR] Invalid input")

    else:
        print("Invalid option, running file upload mode...")
        handle_upload()


# ============================================================
# RUN
# ============================================================
if __name__ == "__main__":
    main()

✓ All libraries loaded successfully!
   🎯 FUKUOKA MQ CHALLENGE SOLVER
   Quaternion Dynamics + PySAT Hybrid Algorithm
   https://www.mqchallenge.org/

    Select mode:
    
      1. 📁 Upload MQ Challenge file (.bz2, .gz, .xz, .txt)
      2. 🏃 Run full challenge simulation (progressive difficulty)
      3. ⚡ Quick test (small problem)
      4. 🎯 Custom test (specify n, m)
    
Enter mode (1/2/3/4) [default=1]: 1

  📁 Upload your MQ Challenge file
  Supported: .bz2, .gz, .xz, .txt, .mq



Saving challenge-6-24-0.bz2 to challenge-6-24-0.bz2

  File: challenge-6-24-0.bz2
  Processing: challenge-6-24-0.bz2
  Raw size: 11454 bytes
  Decompressing .bz2...
  Decompressed: 45343 characters

  File preview (first 500 chars):
  ----------------------------------------
  Galois Field : GF(31)
  Number of variables (n) : 36
  Number of polynomials (m) : 24
  Seed : 0
  Order : graded reverse lex order
  
  *********************
  2 9 21 26 10 6 4 26 27 5 21 23 3 20 6 27 23 6 26 24 5 18 27 11 25 12 4 13 12 16 28 15 12 27 30 2 11 29 6 22 10 17 13 27 7 28 7 25 12 27 1 18 4 12 20 24 2 8 18 8 3 28 13 7 30 9 2 9 6 9 24 7 18 21 10 28 6 13 13 15 10 0 3 12 25 8 23 1 17 12 27 16 1 9 1 12 24 9 9 11 29 25 20 9 24 17 20 22 22 30 15 9 0 6 15 5 16 15 13 20 20 4 15 20 25 12 17 18 9 18 6 0 4 2
  ----------------------------------------

  Parsing MQ Challenge file: challenge-6-24-0.bz2
  ✓ Parsed MQ Problem:
    Field: GF(31)
    Variables (n): 36
    Equations (m): 24
    Parsed equations: 24

  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


  📊 RESULTS SUMMARY

  Problem                 n    m   Result       Time    Heur%   OK
  -----------------------------------------------------------------
  challenge-6-24-0.bz2   36   24      SAT     0.0968    78.7%    ✓

  -----------------------------------------------------------------
  Total: 1 | Solved: 1 (100.0%)
  Total time: 0.10s | Avg heuristic: 78.7%
